### [Performance Measurement and Improvement Techniques]()
Getting a solution is important. But getting it in the fastest way is more important. Learn to check the speed of your code, optimize the code etc.

#### Goal
이미지 처리에서는 초당 많은 작업을 처리하기 때문에 코드가 올바른 솔루션을 제공 할뿐만 아니라 가장 빠른 방식으로 제공되어야합니다. 그래서이 장에서 여러분은

* 코드의 성능을 측정합니다.
* 코드의 성능을 향상시키기위한 몇 가지 팁.
* [cv.getTickCount](https://docs.opencv.org/3.4/db/de0/group__core__utils.html#gae73f58000611a1af25dd36d496bf4487) , [cv.getTickFrequency](https://docs.opencv.org/3.4/db/de0/group__core__utils.html#ga705441a9ef01f47acdc55d87fbe5090c 등의 함수가 표시됩니다 .

OpenCV 외에도 Python 은 실행 시간을 측정하는 데 유용한 모듈 **시간**을 제공합니다 . 다른 모듈 **프로파일**은 코드의 각 함수가 얼마나 많은 시간을 소비했는지, 함수가 얼마나 많이 호출되었는지 등과 같은 코드에 대한 자세한 보고서를 얻는 데 도움이됩니다. 그러나 IPython을 사용하는 경우 이러한 모든 기능이 사용자 친화적 방법. 우리는 몇 가지 중요한 것들을 보게 될 것이며, 자세한 내용은 **추가 자료 섹션** 에서 링크를 확인 하십시오.

#### Measuring Performance with OpenCV
cv.getTickCount function returns the number of clock-cycles after a reference event (like the moment machine was switched ON) to the moment this function is called. So if you call it before and after the function execution, you get number of clock-cycles used to execute a function.

cv.getTickFrequency function returns the frequency of clock-cycles, or the number of clock-cycles per second. So to find the time of execution in seconds, you can do following:

In [7]:
import cv2 as cv
import numpy as np
import sys
sys.path.insert(0, '../common')
import bonghanUtil as u

e1   = cv.getTickCount()
# your code execution
e2   = cv.getTickCount()
time = (e2 - e1)/ cv.getTickFrequency()

print('time is ', time)

time is  2.6e-05


We will demonstrate with following example. Following example apply median filtering with a kernel of odd size ranging from 5 to 49. (Don't worry about what will the result look like, that is not our goal):

In [12]:
img1 = cv.imread('../data/messi5.jpg')

e1 = cv.getTickCount()

for i in range(5,49,2):
    img1 = cv.medianBlur(img1,i)

e2 = cv.getTickCount()
t = (e2 - e1) / cv.getTickFrequency()

print('time is ', time)

time is  2.6e-05


> **Note** You can do the same with time module. Instead of cv.getTickCount, use time.time() function. Then take the difference of two times.

#### Default Optimization in OpenCV
Many of the OpenCV functions are optimized using SSE2, AVX etc. It contains unoptimized code also. So if our system support these features, we should exploit them (almost all modern day processors support them). It is enabled by default while compiling. So OpenCV runs the optimized code if it is enabled, else it runs the unoptimized code. You can use [cv.useOptimized()](https://docs.opencv.org/3.4/db/de0/group__core__utils.html#gafa6d5d04eff341825573ec6c0aa6519f) to check if it is enabled/disabled and [cv.setUseOptimized()](https://docs.opencv.org/3.4/db/de0/group__core__utils.html#ga3c8487ea4449e550bc39575ede094c7a) to enable/disable it. Let's see a simple example.

In [20]:
cv.setUseOptimized(True)
print('UseOptimized : ', cv.useOptimized())
img = cv.imread('../data/messi5.jpg')
%timeit res = cv.medianBlur(img, 49)

cv.setUseOptimized(False)
print('UseOptimized : ', cv.useOptimized())
%timeit res = cv.medianBlur(img, 49)

UseOptimized :  True
19.5 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
UseOptimized :  False
19.3 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


See, optimized median filtering is2x faster than unoptimized version. If you check its source, you can see median filtering is SIMD optimized. So you can use this to enable optimization at the top of your code (remember it is enabled by default).

#### Measuring Performance in IPython
Sometimes you may need to compare the performance of two similar operations. IPython gives you a magic command timeit to perform this. It runs the code several times to get more accurate results. Once again, they are suitable to measure single line codes.

For example, do you know which of the following addition operation is better, ```x = 5; y = x**2, x = 5; y = x*x, x = np.uint8([5]); y = x*x or y = np.square(x) ?``` We will find it with timeit in IPython shell.


In [21]:
cv.setUseOptimized(True)

x = 5

%timeit y = x ** 2
%timeit y = x * x

z = np.uint8([5])

%timeit y = z * z
%timeit y = np.square(z)

cv.setUseOptimized(False)

%timeit res = cv.medianBlur(img, 49)
%timeit y=x ** 2
%timeit y=x * x

z = np.uint8([5])

%timeit y = z * z
%timeit y = np.square(z)


220 ns ± 5.78 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
40.3 ns ± 0.901 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
496 ns ± 5.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
475 ns ± 5.39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
19.6 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
221 ns ± 5.64 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
41.1 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
494 ns ± 7.57 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
474 ns ± 4.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


You can see that, x = 5 ; y = x*x is fastest and it is around 20x faster compared to Numpy. If you consider the array creation also, it may reach upto 100x faster. Cool, right? *(Numpy devs are working on this issue)*

> **Note** Python scalar operations are faster than Numpy scalar operations. So for operations including one or two elements, Python scalar is better than Numpy arrays. Numpy takes advantage when size of array is a little bit bigger.
We will try one more example. This time, we will compare the performance of [cv.countNonZero()](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#gaa4b89393263bb4d604e0fe5986723914) and np.count_nonzero() for same image.

In [28]:
%timeit  z = cv.countNonZero(img[:,:,0]) # Single channel......
%timeit  z = np.count_nonzero(img[:,:,0])

162 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
355 µs ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


See, OpenCV function is nearly 25x faster than Numpy function.

> **Note ** Normally, OpenCV functions are faster than Numpy functions. So for same operation, OpenCV functions are preferred. But, there can be exceptions, especially when Numpy works with views instead of copies.

#### More IPython magic commands
There are several other magic commands to measure the performance, profiling, line profiling, memory measurement etc. They all are well documented. So only links to those docs are provided here. Interested readers are recommended to try them out.

#### Performance Optimization Techniques
There are several techniques and coding methods to exploit maximum performance of Python and Numpy. Only relevant ones are noted here and links are given to important sources. The main thing to be noted here is that, first try to implement the algorithm in a simple manner. Once it is working, profile it, find the bottlenecks and optimize them.

1. Avoid using loops in Python as far as possible, especially double/triple loops etc. They are inherently   slow.
2. Vectorize the algorithm/code to the maximum possible extent because Numpy and OpenCV are optimized for vector operations.
3. Exploit the cache coherence.
4. Never make copies of array unless it is needed. Try to use views instead. Array copying is a costly operation.

Even after doing all these operations, if your code is still slow, or use of large loops are inevitable, use additional libraries like Cython to make it faster.

#### Additional Resources
1. [Python Optimization Techniques](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)
2. Scipy Lecture Notes - [Advanced Numpy](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)
3. [Timing and Profiling in IPython](http://pynash.org/2013/03/06/timing-and-profiling/)  

#### Exercises